In [1]:
pip install fuzzywuzzy

In [2]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
base_names = pd.read_csv('/content/base_names.csv')
name_variations = pd.read_csv('/content/name_variations.csv')

In [13]:
base_names.head()

,Base_Name_ID,Base_Name
0,1,John Smith
1,2,Jennifer Brown
2,3,Michael O'Connor
3,4,Maria Garcia
4,5,Robert Lee


In [14]:
name_variations.head()

,Variation,Matches_With_Base_Name
0,Thomas King,Thomas King
1,ThomasKing,Thomas King
2,Maria Garcia,Maria Garcia
3,MaryLewis,Mary Lewis
4,Nancy W.,Nancy Wright


In [6]:
def preprocess_name(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [15]:
base_names['clean_name'] = base_names['Base_Name'].apply(preprocess_name)
name_variations['clean_name'] = name_variations['Variation'].apply(preprocess_name)

In [16]:
def fuzzy_match_names(base_names, name_variations, threshold=80):
    matches = []
    for name1 in base_names:
        best_match = None
        best_score = 0

        for name2 in name_variations:
            similarity_score = fuzz.token_sort_ratio(name1, name2)

            if similarity_score > best_score and similarity_score >= threshold:
                best_match = name2
                best_score = similarity_score

        matches.append((name1, best_match, best_score))

    return matches

In [20]:
matches = fuzzy_match_names(base_names['clean_name'], name_variations['clean_name'], threshold=80)

In [21]:
results_df = pd.DataFrame(matches, columns=['Name in List 1', 'Best Match in List 2', 'Similarity Score'])

In [22]:
results_df

,Name in List 1,Best Match in List 2,Similarity Score
0,john smith,john smith,100
1,jennifer brown,jennifer brown,100
2,michael oconnor,michael oconnor,100
3,maria garcia,maria garcia,100
4,robert lee,robert lee,100
5,linda johnson,linda johnson,100
6,william davis,william davis,100
7,elizabeth wilson,elizabeth wilson,100
8,david martinez,david martinez,100
9,susan clark,susan clark,100
